In [168]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
from random import randint
import operator


In [169]:
# Number of locations, parts, time steps, and possible quantities respectively
m = 3    # number of locations
h = 3    # number of parts
n = 10    # number of time steps
k = 10     # number of quantities

# Set of possible Locations (L), Parts (P), Time steps (T), and Quantities (Q)
L = list(np.arange(m))
P = list(np.arange(h))
T = list(np.arange(n))
Q = list(np.arange(k))

# Index set of individuals, J
j_max = 100     # max number of individuals we want to start out with
J = np.arange(j_max)


In [170]:
# Individuals [Matrix]
# Creating a random sample of inviduals
Q = np.array([[randint(0,k) for a in T] for b in L])
Q_IJ = np.array([Q for a in J])    # an example population


In [12]:
# Individuals [Tree]
Q_ij = np.arange(k)
from anytree import Node, RenderTree, AsciiStyle, PreOrderIter

Oldest = Node("Oldest")
Inds = [Node("ind:{}".format(j), parent = Oldest) for j in J]
Parts = [[Node("part:{}".format(p), parent = Inds[j]) for p in P] for j in J]
Times = [[[Node("time:{}".format(t), parent = Parts[j][p]) for t in T] for p in P] for j in J]
Quant = [[[[Node("quantity:{}".format(q), 
                 parent = Times[j][p][t]) for q in Q_ij] for t in T] for p in P] for j in J]

for pre, fill, node in RenderTree(Oldest):
    print("%s%s" % (pre, node.name))

ModuleNotFoundError: No module named 'anytree'

In [154]:
c = np.array([randint(4,8) for a in L]).T     # cost vector, assuming fixed cost by location
c = c.reshape(1,3)
c.shape


(1, 3)

In [39]:
#print(RenderTree(Oldest, style=AsciiStyle()).by_attr())

In [254]:
# Fitness -- for now, only considering min{COST}
# this function takes in AN INDIVIDUAL of a population, Q, evaluates it, and returns a metric the evaluation/fitnesss
def fitness(Q):
    # Cost matrix (or vector or constant) to map quantities to cost
    c = np.array([randint(4,8) for a in L]).reshape(1,m)     # cost vector, assuming fixed cost by location
    #C = (c*np.eye(m,n).T).T

    # Availability and associated Cost
    a = 0.95  # availability level
    R = np.array([[5 for a in T] for b in L])    # Requirements matrix, ie. how much is 'required' by each location
    C_av = sum(sum(a*(np.dot(c,R))))                 # cost associated with having a required availability

    # Cost associated with choice of Individual
    C_ind = sum(sum(np.dot(c,np.array(Q).reshape(m,n))))
                              # percentage difference between cost of availability and cost of choice
                              # Goal = get cost difference VERY close to zero --> our individuals ~ meet requirements
    return (C_ind - C_av)/C_av       

In [255]:
def gene_fitness(Q):  
    Q.reshape(m*n)
    

In [256]:
def ranking(Q_IJ):
    pairs = []
    for a in J:
        fit = abs(fitness(Q_IJ[a]))
        pairs.append((fit, Q_IJ[a]))
    pairs.sort(key=operator.itemgetter(0))
    return pairs


In [257]:
# Evolutionary probabilities
p_sel = 0.7
p_cross = 0.3
P_mut = 0.3

In [258]:
# Evolutionary Operators
# returns array of pairs of (fitness(individual), individual)
def tournament_selection(Q_IJ, k):
    Q_IJ = np.array(Q_IJ)
    #N = int(p_sel*len(J))         # selecting 0.7 percent of the population to be selected as parents
    parents = []                 # start out with no parents to compete
    while len(parents) <= len(Q_IJ):    
        for b in np.arange(len(Q_IJ)):
            if type(Q_IJ[b]) == tuple:
                ind1 = Q_IJ[randint(1, len(Q_IJ)-1)][1]
                ind2 = Q_IJ[randint(1, len(Q_IJ)-1)][1]
            else:
                ind1 = Q_IJ[randint(1, len(Q_IJ)-1)]
                ind2 = Q_IJ[randint(1, len(Q_IJ)-1)]
            players = [ind1, ind2]
            best = (max(abs(fitness(ind1)), 
                        abs(fitness(ind2))),
                    players[np.argmax([abs(fitness(ind1)), 
                                       abs(fitness(ind2))])])
            parents.append(best)
    return np.array(parents)

In [259]:
# Input = array of pairs (fitness(individual), individual)
# Output = array of individuals that had their genes crossed over
def crossover(parents):
    children = []
    while len(children) <= len(parents): #int(p_cross*j_max):
        for a in np.arange(len(parents)):

            parent1 = parents[np.random.randint(0,len(parents))][1]
            parent2 = parents[np.random.randint(0,len(parents))][1]

            crosspoint = np.random.randint(0,m*n)      # don't know where in the Time series this lands
            Gene1 = list(parent1.reshape(m*n))
            Gene2 = list(parent2.reshape(m*n))
            child1 = np.array(Gene1[0:crosspoint] + Gene2[crosspoint:])   
            child2 = np.array(Gene2[0:crosspoint] + Gene1[crosspoint:])
            child1 = child1.reshape(m,n)
            child2 = child2.reshape(m,n)
            children.append(child1)
            children.append(child2)
    return children

In [287]:
# Input = array of pairs (fitness(individual), individual) AND 'b', the percentage to select
# Output = new array of 'elite' pairs (fitness(individual), individual)

def elitism(parents, b):   # of best individuals to pass on
    return ranking(parents)[0:int(b*len(parents))]


In [261]:
# mutation
# Here, we convert the forecasts for parts at each location, l,  of each invidual to either 1's or 0's;
# 1 = forecast costs less , 0 = forecast costs more
# The goal here is to flip genes (forecasts) across each individual so that each has the most ones 

def mutation(children):
    for child in children:
        child[1] + np.random.randint(-1,1)
    return children

# Non-DEAP Algorithm

In [289]:
# Number of locations, parts, time steps, and possible quantities respectively
m = 3    # number of locations
h = 3    # number of parts
n = 10    # number of time steps
k = 10     # number of quantities

# Set of possible Locations (L), Parts (P), Time steps (T), and Quantities (Q)
L = list(np.arange(m))
P = list(np.arange(h))
T = list(np.arange(n))
Q = list(np.arange(k))

# Index set of individuals, J
j_max = 100     # max number of individuals we want to start out with
J = np.arange(j_max)


In [290]:
# Individuals [Matrix]
# Creating a random sample of inviduals
Q = np.array([[randint(0,k) for a in T] for b in L])
Q_IJ = np.array([Q for a in J])    # an example population
len(Q_IJ)

100

In [291]:
# The Algorithm
Q = np.array([[randint(0,k) for a in T] for b in L])
Q_IJ = np.array([Q for a in J])    # an example population


In [288]:
len(elitism(tournament_selection(Q_IJ,2), 0.5))

ValueError: cannot reshape array of size 2 into shape (3,10)

In [292]:

def Genetic_Algorithm(P_init, i):
    i_max = i # number of iterations/generations
    I = np.arange(i_max)
    for i in I:       
        winners = tournament_selection(P_init, 2)
        selected = elitism(winners, 0.5) # Elitism
        crossed = crossover(selected)
        crossed = ranking(crossed)[0:int(0.5*len(crossed))] # China's one-child policy
        Q_IJ = list(np.array([mutation(crossed)[a][1] for a in np.arange(int(len(crossed)))]))[:int(0.5*len(crossed))]
    return Q_IJ

In [294]:
print(Genetic_Algorithm(Q_IJ, 5)[2])
#print(Q_IJ[2])

ValueError: cannot reshape array of size 2 into shape (3,10)

In [158]:
np.dot(c,Q)

array([[ 51, 126,  48, 163,  71,  92, 104, 103, 154, 117]])

## DEAP Algorithm

In [317]:
import random

from deap import base
from deap import creator
from deap import tools


In [318]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)


C:\Users\jek12\Anaconda3\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\Users\jek12\Anaconda3\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [319]:
def indiv(k):
    return np.array([[randint(0,k) for a in T] for b in L]).reshape(m*n)


In [ ]:
toolbox = base.Toolbox()
# Attribute generator 
toolbox.register("attr_bool", indiv, 10)
# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_bool, 100)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


In [320]:
def evalOneMax(indiv):
    return sum(indiv)

toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=2)


In [324]:
def main():
    pop = toolbox.population(n=300)
    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
    # CXPB  is the probability with which two individuals
    #       are crossed
    #
    # MUTPB is the probability for mutating an individual
    CXPB, MUTPB = 0.5, 0.2
    # Extracting all the fitnesses of 
    fits = [ind.fitness.values[0] for ind in pop]
    # Variable keeping track of the number of generations
    g = 0
    
    # Begin the evolution
    while max(fits) < 100 and g < 1000:
        # A new generation
        g = g + 1
        print("-- Generation %i --" % g)
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))
       # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        pop[:] = offspring

        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]
        
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5
        
        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)
    return pop

In [325]:
main()


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [326]:
print(main())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.
